# 5. トレーニング設定

In [ ]:

# 5. トレーニング設定
### 5.1 トレーニング引数設定
# Split dataset into training and evaluation sets
dataset_size = len(tokenized_dataset)
indices = np.random.permutation(dataset_size)
split_idx = int(dataset_size * 0.8)
train_dataset = tokenized_dataset.select(indices[:split_idx])
# Limit evaluation dataset size
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+50])  # Maximum 50 samples

logging.info(f"Training dataset size: {len(train_dataset)}")
logging.info(f"Evaluation dataset size: {len(eval_dataset)}")

# Disable wandb via environment variable
os.environ["WANDB_DISABLED"] = "true"

# Update training arguments
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,  
    num_train_epochs=30,
    learning_rate=8e-5,
    weight_decay=0.06,
    warmup_ratio=0.25,
    lr_scheduler_type="cosine_with_restarts",
    evaluation_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=20,
    gradient_accumulation_steps=8,
    max_steps=-1,
    disable_tqdm=False,
    logging_dir=LOG_OUTPUT_DIR,   
    logging_strategy="steps",
    logging_steps=50,
    no_cuda=False,
    dataloader_num_workers=1,
    report_to=[],
    run_name=None,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_checkpointing=True,
    max_grad_norm=0.5,
    dataloader_pin_memory=True,
    save_total_limit=2,
    fp16=True,
    optim="adamw_torch_fused",
    eval_accumulation_steps=4,
    load_best_model_at_end=True,
    metric_for_best_model="combined_score",
)



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードセクションについて、ソクラテス式チャットボットのトレーニングを例に説明していきます。

### 1. データセットの分割

```python
dataset_size = len(tokenized_dataset)
indices = np.random.permutation(dataset_size)
split_idx = int(dataset_size * 0.8)
train_dataset = tokenized_dataset.select(indices[:split_idx])
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+50])
```

これは、データセットを「トレーニング用」と「評価用」に分けている部分です。

- 全体の80%をトレーニング用に使用
- 残りの中から最大50個のサンプルを評価用に使用

例えば、1000個の対話データがあった場合：
- トレーニング用：800個の対話
- 評価用：50個の対話
を使用することになります。

### 2. トレーニング設定（TrainingArguments）

主要な設定を説明します：

#### 基本設定
```python
num_train_epochs=30,  # 30回繰り返してトレーニング
learning_rate=8e-5,   # 学習率（どのくらい大きく更新するか）
```

#### 学習の進め方
```python
warmup_ratio=0.25,  # 最初の25%は徐々に学習率を上げていく
lr_scheduler_type="cosine_with_restarts",  # 学習率を周期的に変化させる
```

これは、ソクラテス式の対話の特徴（質問の仕方、応答の仕方）を段階的に学習させるための設定です。

#### 評価と保存
```python
evaluation_strategy="steps",
eval_steps=20,        # 20ステップごとに評価
save_strategy="steps",
save_steps=20,        # 20ステップごとに保存
```

例えば：
- 20回の対話トレーニングが終わるごとに
- モデルが「なぜそう考えるのですか？」「それはどういう意味でしょうか？」といった
  ソクラテス式の質問ができているかを評価

#### バッチ処理設定
```python
per_device_train_batch_size=2,  # 一度に2つの対話をトレーニング
gradient_accumulation_steps=8,   # 8回分まとめて更新
```

メモリの制約上、一度に処理できる対話数を制限しています。

#### 評価指標
```python
metric_for_best_model="combined_score",  # 総合評価スコアで判断
load_best_model_at_end=True,            # 最も良い結果のモデルを保存
```

この「combined_score」には以下のような要素が含まれます：
- 質問の適切さ（「なぜ」「どのように」などの問いかけ）
- 対話の流れの自然さ
- ソクラテス式の特徴的な言い回しの使用

これらの設定により、ソクラテスのように：
- 適切なタイミングで質問を投げかけ
- 相手の考えを深めるような対話を行う
能力を効率的に学習させることを目指しています。

        
</div>
    </details>
</div>


In [ ]:

### 5.2 データローダーとコレータ設定
# Modify data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

データローダーとコレータ設定について、ソクラテス式チャットボットの例を用いて説明します。

### データコレータ（Data Collator）とは？

データコレータは、バラバラの対話データを機械学習用の形式に整形するツールです。例えるなら、様々な長さの対話を同じサイズの「お弁当箱」に詰めるようなものです。

```python
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,    # テキストを数値に変換するツール
    mlm=False,             # マスク言語モデリングを使用しない
    pad_to_multiple_of=8   # データの長さを8の倍数に揃える
)
```

### 具体例で説明

例えば、以下のような異なる長さの対話があるとします：

1. 短い対話：
```
ユーザー：「幸せとは何でしょうか？」
モデル：「幸せとは何だとお考えですか？」
```

2. 長い対話：
```
ユーザー：「幸せとは何でしょうか？」
モデル：「幸せとは何だとお考えですか？」
ユーザー：「家族と過ごす時間だと思います」
モデル：「なぜ家族と過ごす時間が幸せだとお感じになるのでしょうか？」
```

### データコレータの役割

1. **長さの統一**
   - `pad_to_multiple_of=8`は、すべての対話データの長さを8の倍数に調整します
   - 短い対話には特殊なパディング（埋め合わせ）トークンを追加
   - これにより、GPUでの計算効率が上がります

2. **入力形式の統一**
   - 対話をモデルが理解できる数値形式（トークン）に変換
   - 例：「幸せ」→ [2345, 6789]（数値の例）

3. **バッチ処理の準備**
   - 複数の対話を一度に処理できるように整形
   - メモリ使用を最適化

### mlm=Falseの意味

- `mlm=False`は、このモデルが「マスク言語モデリング」を使用しないことを示します
- ソクラテス式チャットボットでは、文章の一部を隠して予測する必要はなく、
  対話の流れを自然に学習させたいため、これをFalseに設定しています

このように、データコレータは、様々な形式の対話データを、モデルが効率的に学習できる形に整形する重要な役割を果たしています。

        
</div>
    </details>
</div>


In [ ]:

### 5.3 トレーニング実行と例外処理
# Start training
logging.info("Starting training...")
try:
    checkpoint_dir = MODEL_OUTPUT_DIR  
    resume_from_checkpoint = None
    
    # Check if running in Kaggle environment
    is_kaggle = os.path.exists('/kaggle/working')
    
    # Checkpoint status and processing
    if os.path.exists(checkpoint_dir):
        print("\nChecking checkpoint status...")  
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
        if checkpoints:
            # Get latest checkpoint
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
            checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
            print(f"Found latest checkpoint: {latest_checkpoint}") 
            
            # Check checkpoint status
            state_path = os.path.join(checkpoint_path, "trainer_state.json")
            if os.path.exists(state_path):
                with open(state_path, 'r') as f:
                    state = json.load(f)
                current_epoch = state.get('epoch', 0)
                print(f"\nCurrent training status:")  
                print(f"Current epoch: {current_epoch}")  
                print(f"Target epochs: {training_args.num_train_epochs}")  
                
                # Exit safely if completed
                if current_epoch >= training_args.num_train_epochs - 0.1:
                    print("\n" + "="*50)
                    print("IMPORTANT NOTICE:")
                    print(f"Training has already been completed at epoch {current_epoch}!")
                    print(f"Target epochs was {training_args.num_train_epochs}")  
                    print(f"Trained model is available at: {checkpoint_dir}")
                    print("="*50 + "\n")
                    logging.info("Training has already been completed. Exiting to protect existing model.")
                    logging.info(f"Trained model is available at: {checkpoint_dir}")
                    exit(0)
            else:
                logging.warning("Invalid checkpoint state found. Please check manually.")
                logging.warning(f"Checkpoint directory: {checkpoint_dir}")
                if not is_kaggle:  
                    user_input = input("Do you want to continue and overwrite? (yes/no): ")
                    if user_input.lower() != 'yes':
                        logging.info("Aborting to protect existing data.")
                        exit(0)
        else:
            logging.warning("Checkpoint directory exists but no checkpoints found.")
            if not is_kaggle:  
                user_input = input("Do you want to continue and overwrite the directory? (yes/no): ")
                if user_input.lower() != 'yes':
                    logging.info("Aborting to protect existing data.")
                    exit(0)

    # Start training (or resume)
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    logging.info("Training completed successfully!")


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



トレーニング実行と例外処理の部分について、ソクラテス式チャットボットの例を用いて説明します。

### 1. トレーニングの開始準備

```python
logging.info("Starting training...")
try:
    checkpoint_dir = MODEL_OUTPUT_DIR  
    resume_from_checkpoint = None
```

これは、トレーニングを始める準備をする部分です。チェックポイント（途中経過の保存）を保存するディレクトリを設定します。

### 2. チェックポイントの確認システム

チェックポイントとは、トレーニングの途中経過を保存したものです。例えば：
- 10エポック目でのソクラテス式の質問の仕方
- 20エポック目での対話の自然さ
などの学習状態を保存しています。

```python
if os.path.exists(checkpoint_dir):
    print("\nChecking checkpoint status...")  
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
```

### 3. 最新のチェックポイントの確認

```python
if checkpoints:
    # 最新のチェックポイントを見つける
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
```

例えば：
- checkpoint-1000（10エポック目）
- checkpoint-2000（20エポック目）
- checkpoint-3000（30エポック目）
がある場合、checkpoint-3000を最新として選びます。

### 4. トレーニング状態の確認

```python
state_path = os.path.join(checkpoint_path, "trainer_state.json")
if os.path.exists(state_path):
    with open(state_path, 'r') as f:
        state = json.load(f)
    current_epoch = state.get('epoch', 0)
```

これは、以下のような情報を確認します：
- 現在のエポック数（何周目の学習か）
- 目標のエポック数（何周学習する予定か）

### 5. トレーニング完了チェック

```python
if current_epoch >= training_args.num_train_epochs - 0.1:
    print("IMPORTANT NOTICE:")
    print(f"Training has already been completed at epoch {current_epoch}!")
```

例えば：
- 目標が30エポックで
- すでに30エポック完了している場合
- 「トレーニング済み」として処理を終了

### 6. 安全確認と再開の選択

```python
if not is_kaggle:  
    user_input = input("Do you want to continue and overwrite? (yes/no): ")
    if user_input.lower() != 'yes':
        logging.info("Aborting to protect existing data.")
        exit(0)
```

これは、既存のトレーニング結果を誤って上書きしないための安全装置です。
例えば：
- すでに良い感じのソクラテス式の対話ができるモデルがある場合
- 誤って上書きしないように確認を求めます

### 7. トレーニングの実行

```python
trainer.train(resume_from_checkpoint=resume_from_checkpoint)
logging.info("Training completed successfully!")
```

最後に、実際のトレーニングを開始します：
- 新規トレーニングの場合：最初から開始
- 途中再開の場合：チェックポイントから再開

このように、このコードは：
1. トレーニングの状態を確認
2. 既存のモデルを保護
3. 適切な開始位置を決定
という重要な役割を果たしています。

        
</div>
    </details>
</div>


In [ ]:
### 5.4 モデル保存と設定エクスポート
    # Save settings (as JSON)
    def convert_to_serializable(obj):
        if isinstance(obj, set):
            return list(obj)
        elif isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [convert_to_serializable(x) for x in obj]
        return obj

    # Convert each setting
    training_args_dict = convert_to_serializable(training_args.to_dict())
    lora_config_dict = convert_to_serializable(lora_config.to_dict())

    config_dict = {
        "model_name": model_name,
        "training_args": training_args_dict,
        "lora_config": lora_config_dict,
        "bnb_config": {
            "load_in_4bit": bnb_config.load_in_4bit,
            "bnb_4bit_use_double_quant": bnb_config.bnb_4bit_use_double_quant,
            "bnb_4bit_quant_type": bnb_config.bnb_4bit_quant_type,
            "bnb_4bit_compute_dtype": str(bnb_config.bnb_4bit_compute_dtype),
        }
    }
    
    # Save configurations
    with open(os.path.join(training_args.output_dir, "training_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_dict, f, indent=2, ensure_ascii=False)
    
    # Save model and settings
    trainer.save_model()
    model.config.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    logging.info("Model and configuration saved successfully!")

except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    raise


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



モデル保存と設定エクスポートの部分について、ソクラテス式チャットボットの例を用いて説明します。

### 1. データ変換用の関数定義

```python
def convert_to_serializable(obj):
    if isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(x) for x in obj]
    return obj
```

これは、モデルの設定を保存可能な形式に変換する関数です。

例えば：
- 特殊な質問パターンのセット `{'なぜ', 'どのように', 'どう考えますか'}` を
- 保存可能な配列 `['なぜ', 'どのように', 'どう考えますか']` に変換

### 2. トレーニング設定の変換

```python
training_args_dict = convert_to_serializable(training_args.to_dict())
lora_config_dict = convert_to_serializable(lora_config.to_dict())
```

トレーニングの設定を保存用に変換します。例えば：
- 学習率
- エポック数
- バッチサイズ
などの設定値を保存可能な形式に変換します。

### 3. 設定情報の整理

```python
config_dict = {
    "model_name": model_name,        # 使用したベースモデルの名前
    "training_args": training_args_dict,  # トレーニング設定
    "lora_config": lora_config_dict,      # LoRA（効率的な学習方法）の設定
    "bnb_config": {                       # メモリ効率化の設定
        "load_in_4bit": bnb_config.load_in_4bit,
        "bnb_4bit_use_double_quant": bnb_config.bnb_4bit_use_double_quant,
        "bnb_4bit_quant_type": bnb_config.bnb_4bit_quant_type,
        "bnb_4bit_compute_dtype": str(bnb_config.bnb_4bit_compute_dtype),
    }
}
```

これは、モデルの全設定をまとめる部分です。例えば：
- ベースモデル：「gemma-2b-jpn-it」
- トレーニング設定：30エポック、学習率8e-5など
- LoRA設定：効率的な学習のための特別な設定
- メモリ効率化設定：限られたGPUメモリでも動作するための設定

### 4. 設定の保存

```python
with open(os.path.join(training_args.output_dir, "training_config.json"), "w", encoding="utf-8") as f:
    json.dump(config_dict, f, indent=2, ensure_ascii=False)
```

設定をJSON形式で保存します。これにより：
- 後で設定を確認できる
- 同じ設定で再トレーニングできる
- トラブル時の参照が可能

### 5. モデルと関連ファイルの保存

```python
trainer.save_model()
model.config.save_pretrained(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)
```

これは実際のモデルと必要なファイルを保存します：
- 学習済みモデル（ソクラテス式の対話ができるようになったモデル）
- モデルの設定ファイル
- トークナイザー（テキストを数値に変換するツール）

### 6. エラー処理

```python
except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    raise
```

保存中にエラーが発生した場合の対策です。例えば：
- ディスク容量不足
- 保存先へのアクセス権限がない
などの問題が発生した場合に、エラーメッセージを記録します。

このように、この部分は学習したモデルと設定を安全に保存し、後で再利用できるようにする重要な役割を果たしています。

        
</div>
    </details>
</div>
